# What classes would we need for a backtrading system?

* Trade and TradeWrapper
* Broker
* TradingSystem


In [1]:
from collections import namedtuple

"""
The Trade class is simply a namedtuple that has the fields of a trade in it, like date, ticker, price, and shares.
We don't normally call it directly; instead, we use TradeWrapper.
b_s is buy/sell and is a single letter 'b' or 's'
o_c is open/close and is a single letter 'o' or 'c'
net_shares is the same as shares for buy. It's -1 * shares for a sell. (So Buy 100 shares IBM -> net_shares = 100. Sell 1 share IBM -> net_shares = -1
"""
# Just a namedtuple.
Trade = namedtuple("Trade", "date ticker price shares b_s o_c net_shares commission")

In [4]:
from datetime import datetime
from typing import Union
# from trade import Trade

class TradeWrapper:
    def __init__(self, date: datetime, ticker: str, price: float, shares: Union[float, int], buy_or_sell: str = 'b',
                 open_or_close: str = 'o', commission: float = 0.0):
        self._date = date
        self._ticker = ticker.upper()
        self._price = price
        self._shares = shares
        self._b_s = buy_or_sell.lower()[0]
        self._o_c = open_or_close.lower()[0]
        self._commission = commission
        self._net_shares = shares if self.is_buy else (-1) * shares # so sell 10 shares makes _net_shares -10.
        self._trade = Trade(self.date, self.ticker, self.price, self._net_shares, self._b_s, self._o_c, self._net_shares, self._commission)

    @property
    def trade(self):
        return self._trade

    @property
    def date(self):
        return self._date

    @property
    def ticker(self):
        return self._ticker

    @property
    def price(self):
        return self._price

    @property
    def shares(self):
        return self._shares

    @property
    def is_buy(self):
        return self._b_s == 'b'

    @property
    def is_sell(self):
        return not self.is_buy

    @property
    def is_open(self):
        return self._o_c == 'o'

    @property
    def is_close(self):
        return not self.is_open

    @property
    def net_shares(self):
        return self._net_shares

    def __str__(self) -> str:
        b_s = 'bought' if self.is_buy else 'sold'
        o_c = 'open' if self.is_open else 'close'
        return  f'On {self.date}, {b_s} {self.shares} shares of {self.ticker} to {o_c}'


In [6]:
import logging
from datetime import datetime
import pandas as pd
from pathlib import Path
from typing import Union, List
#from trade import Trade
#from TradeWrapper import TradeWrapper

"""
The Broker class does items related to a Brokerage firm, like:
  - Maintain a cash balance
  - record trades in a portfolio
  - tell us how many shares we can buy
  - reports opening and closing cash
"""
class Broker:
    def __init__(self, ticker: str, starting_bal: Union[float, int] = 10000):
        self.logger = logging.getLogger('StockProject.Broker')
        self._ticker = ticker
        self._df = self.read_prices()
        self._starting_bal = starting_bal
        self._cash = starting_bal
        self._trade_wrappers = []
        self.init_portfolio(starting_bal)

    def init_portfolio(self, starting_bal: Union[float, int] = None):
        """
        Empty out the trade list and set the cash to the starting balance
        :param starting_bal:
        :return:
        """
        self._cash = starting_bal or self._starting_bal
        self._trade_wrappers = []

    @property
    def ticker(self):
        return self._ticker

    @property
    def trades(self) -> List:
        all_trades = [t.trade for t in self._trade_wrappers] # Turn list of TradeWrappers into list of Trades
        return all_trades

    @property
    def trades_as_df(self) -> pd.DataFrame:
        df = pd.DataFrame(data=self.trades)
        return df

    @property
    def cash(self):
        return self._cash

    # methods involving the portfolio
    def add_tradeWrapper(self, trade_wrapper: TradeWrapper):
        """
        Reduce (or raise) cash by the amount of the stock purchase (or sale)
        Add the trade to the list of trades.
        :param trade_wrapper    wrapped trade object
        :return:
        """
        trade = trade_wrapper.trade
        self._cash = self._cash - trade.net_shares * trade.price - trade.commission
        self._trade_wrappers.append(trade_wrapper)

    def portfolio(self, report_all=False):
        """
        Return the portfolio as a dataframe.
        :param report_all: if True, report even if net shares are 0. If False, don't report if the net shares are 0.
        :return:
        """
        port_df = self.trades_as_df
        if port_df.empty:
            return port_df
        netted_df = port_df.groupby(port_df.ticker).sum()
        if report_all:
            return netted_df
        return netted_df[netted_df['net_shares'] != 0]

    def trade_report(self):
        for trade_no, trade_wrapper in enumerate(self._trade_wrappers, start=1):
            self.logger.info(f'{trade_no:3d}. {trade_wrapper}')

    def report(self):
        """
        Generate a report of our portfolio balances and final cash.
        :return:
        """
        df = self.portfolio()
        self.logger.info(f'Cash report\nStarting balance: ${self._starting_bal:.2f}\nEnding balance: ${self.cash_balance():.2f}')
        self.logger.info('Trades')
        self.trade_report()
        if len(df):
            df.drop(columns=['price', 'shares'], inplace=True)
            self.logger.info(f'Portfolio report\nPortfolio\n{df}\n')
        else:
            self.logger.info(f'Portfolio report\nEmpty portfolio\n')
        pass # TODO: Summarize the trade gains or losses with count, average, max loss, max gain
        pass # TODO: Summarize the percentage gain for this system


    def tickers(self) -> List:
        """
        Report the tickers
        :return: List of unique tickers.
        """
        return self.trades_as_df.ticker.unique().tolist()

    def cash_balance(self) -> float:
        return self._cash

    def is_flat(self) -> bool:
        """
        Look at the net portfolio
        :return: True iff net portfolio is empty.
        """
        net_df = self.portfolio(report_all=False)
        return len(net_df) == 0

    # methods involving the prices DB
    def row_count(self):
        return len(self._df)

    def closing_prices(self, closing_col_name: str = 'Close') -> pd.DataFrame:
        """
        return the closing prices as a ddtaframe.
        :param closing_col_name: column name to use to slice the closing prices, e.g., 'Adj Close'
        :return: dataframe of the requested column
        """
        return self._df[closing_col_name]

    def price_at_date(self, date: Union[datetime, str],  format: str = '%Y-%m-%d') -> pd.DataFrame:
        """

        :param date:
        :param format:   format of the date; see https://docs.python.org/3/library/datetime.html?highlight=strptime#strftime-and-strptime-behavior
        :return:
        """
        dt = date if isinstance(date, datetime) else datetime.strptime(date, format)
        try:
            row = self._df[self._df.index == dt]
            ans = (self.ticker, date, row['Close'][0], row['Adj Close'][0])
            return ans
        except IndexError:
            self.logger.warning(f'Could not find date {date}! Returning None.')
            return None

    def read_prices(self):
        raise NotImplementedError('must be implemented in subclass')

    def coerce_to_date(self, date_col: str = 'Date', format: str = '%Y-%m-%d'):
        """
        Coerce the date column to a string for the internal dataframe.
        :param date_col: string of the column name, default 'Date'
        :param format:   format of the date; see https://docs.python.org/3/library/datetime.html?highlight=strptime#strftime-and-strptime-behavior
        :return:
        """
        self._df[date_col] = pd.to_datetime(self._df[date_col], format=format)

    def set_index(self, index_col: str = 'Date'):
        """
        Set the index column for the internal dataframe.
        :param index_col: string of the column name, default 'Date'
        :return: None
        """
        self._df.set_index(keys=index_col, inplace=True)
        return

    def head(self, how_many: int = 5):
        """
        Print the first n records of the dataframe.

        :param how_many: how many to print
        :return: None
        """
        self.logger.info(self._df.head(how_many))

    def graph(self):
        """
        Display a graph of the prices with the system buys and sells.
        TODO: Add a graph for the technical indicators.
        :return:
        """
        pass # TODO: Put your code here!


class Static_Broker(Broker):
    def __init__(self, ticker: str):
        super().__init__(ticker)
        self.logger.debug('instantiating prices in Static_Broker')
        self.coerce_to_date('Date', format='%Y-%m-%d')
        self.set_index('Date')

    def read_prices(self):
        fn = f'{self.ticker}.CSV'
        p = Path(fn)
        if not p.is_file():
            self.logger.error(f'cannot find file {fn}.')

        ans = pd.read_csv(p)
        self.logger.debug(f'read {len(ans)} records from {fn}')
        return ans

In [8]:
import logging
from datetime import datetime
from typing import Union

import pandas as pd

#from TradeWrapper import TradeWrapper
#from broker import Broker

"""
The TradingSystem class implements trading systems.
Known subclasses: BuyAndHoldSystem, SmaTrendSystem
  - You initialize it with a Broker (either dynamic or static)
  - Your subclass implements run(), which tells it when to buy and sell
  - tell us how many shares we can buy
  - reports opening and closing cash
"""
class TradingSystem:
    def __init__(self, broker: Broker):
        self.logger = logging.getLogger('StockProject.TradingSystem')
        self._broker = broker
        self._prices_df = pd.DataFrame(data=broker.closing_prices())

    def run(self):
        raise NotImplementedError('must be implemented in subclass')

    def buy_to_open(self, ticker: str, date: str, price: float, shares: Union[int, float]):
        trade1 = TradeWrapper(date=date, ticker=ticker, price=price, shares=shares, buy_or_sell='Buy',
                              open_or_close='open')
        self._broker.add_tradeWrapper(trade1)

    def sell_to_close(self, ticker: str, date: str, price: float, shares: Union[int, float]):
        trade1 = TradeWrapper(date=date, ticker=ticker, price=price, shares=shares, buy_or_sell='Sell',
                              open_or_close='close')
        self._broker.add_tradeWrapper(trade1)

    def sell_to_open(self, ticker: str, date: str, price: float, shares: Union[int, float]):
        raise NotImplementedError('must be implemented if you want to sell short') # TODO

    def buy_to_close(self, ticker: str, date: str, price: float, shares: Union[int, float]):
        raise NotImplementedError('must be implemented if you want to sell short') # TODO

    def add_indicator(self, indicator_type: str = 'SMA', indicator_period: int = 20):
        """
        Add an indicator to the dataframe
        :param indicator_type:
        :param indicator_period:
        :return:
        """
        indicator_type = indicator_type.upper()
        new_col = f'{indicator_type}_{indicator_period}' # Default is SMA_20
        if indicator_type == 'SMA':
            self._prices_df[new_col] = self._prices_df.rolling(window=indicator_period).mean()
        elif indicator_type == 'yourindicator':
            pass # TODO add your indicators here
        else:
            self.logger.warning(f'Cannot add an indicator type of {indicator_type}')
        return

    def close_positions(self, date: datetime, shares: Union[int, float]):
        """
        Close any open positions.
        :param date:
        :param shares:
        :return:
        """
        if self._broker.is_flat():
            return
        closes = self._broker.closing_prices()
        ticker, date, _, adj_close = self._broker.price_at_date(date=date)
        self.sell_to_close(ticker=ticker, date=date, price=adj_close, shares=shares)


class BuyAndHoldSystem(TradingSystem):
    def __init__(self, broker: Broker):
        super().__init__(broker)
        self.logger.debug('Starting buy-and-hold system.')

    def run(self):
        """
        Simply buy on the first day and sell on the last.
        :return:
        """
        closes = self._broker.closing_prices()
        shares = int(self._broker.cash_balance() / closes[0])
        ticker, date, _, adj_close = self._broker.price_at_date(closes.index[0])
        self.buy_to_open(ticker=ticker, date=date, price=adj_close, shares=shares)
        last = len(closes) - 1
        self.close_positions(date=closes.index[last], shares=shares)


class SmaTrendSystem(TradingSystem):
    def __init__(self, broker: Broker, sma_period: int = 20):
        super().__init__(broker)
        self.logger.debug('Starting SMA trend system.')
        indicator_type = 'SMA'

        self._indicator1 = f'{indicator_type}_{sma_period}' # Default is SMA_20
        self.add_indicator(indicator_type=indicator_type, indicator_period=sma_period)


    def run(self):
        """
        Simply buy on the first day and sell on the last.
        :return:
        """
        for row in self._prices_df.itertuples(name='Prices'):
            sma = getattr(row, self._indicator1) # Better than row.SMA_20

            if self._broker.is_flat():
                if row.Close > sma:
                    # We're flat and the Price crossed over SMA, so buy
                    ticker, date, _, _ = self._broker.price_at_date(row.Index)
                    shares = int(self._broker.cash_balance() / row.Close)
                    self.buy_to_open(ticker=ticker, date=date, price=row.Close, shares=shares)
            else:
                # We are long and will test to see if we should close the position
                if row.Close < sma:
                    # Price crossed under SMA, so sell
                    ticker, date, _, _ = self._broker.price_at_date(row.Index)
                    self.sell_to_close(ticker=ticker, date=date, price=row.Close, shares=shares)

        # After going through the DB, if we are not flat, then go flat.
        closes = self._broker.closing_prices()
        last = len(closes) - 1
        self.close_positions(date=closes.index[last], shares=shares)

# TODO: Add your own subclass of TradingSystem

In [12]:
import logging

#from broker import Static_Broker
#from tradingSystem import BuyAndHoldSystem, SmaTrendSystem

logger = logging.getLogger('StockProject.main')
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger.debug('Starting!')
b = Static_Broker('TSLA')

# Run 1: Buy and hold
s = BuyAndHoldSystem(broker=b)
s.run()
b.report()

# Run 2: SMA
b.init_portfolio() # set back to initial
s = SmaTrendSystem(broker=b, sma_period=50)
s.run()
b.report()

# Run 3: Your own system
b.init_portfolio() # set back to initial
# TODO: put your call here

logger.info('Done.')


2022-03-15 15:59:48,862 - StockProject.main - DEBUG - Starting!
2022-03-15 15:59:48,865 - StockProject.Broker - DEBUG - read 252 records from TSLA.CSV
2022-03-15 15:59:48,865 - StockProject.Broker - DEBUG - instantiating prices in Static_Broker
2022-03-15 15:59:48,867 - StockProject.TradingSystem - DEBUG - Starting buy-and-hold system.
2022-03-15 15:59:48,872 - StockProject.Broker - INFO - Cash report
Starting balance: $10000.00
Ending balance: $16453.75
2022-03-15 15:59:48,873 - StockProject.Broker - INFO - Trades
2022-03-15 15:59:48,873 - StockProject.Broker - INFO -   1. On 2020-12-28 00:00:00, bought 15 shares of TSLA to open
2022-03-15 15:59:48,874 - StockProject.Broker - INFO -   2. On 2021-12-27 00:00:00, sold 15 shares of TSLA to close
2022-03-15 15:59:48,874 - StockProject.Broker - INFO - Portfolio report
Empty portfolio

2022-03-15 15:59:48,874 - StockProject.TradingSystem - DEBUG - Starting SMA trend system.
2022-03-15 15:59:49,152 - StockProject.Broker - INFO - Cash report


In [10]:
pwd

'C:\\Users\\rajah\\OneDrive\\Documents\\Learning and Teaching\\ELVTR\\Module 12'